# Scaling and reorganizing heating/cooling demand data.

This script reads, normalises, and reorganises the raw ArchetypeBuildingModel output
for Mopo WP5.
Depending on the exact needs, these scripts might need quite a bit of tinkering later on as well, but let's do something for now.

## Julia environment setup

In [ ]:
## Julia environment setup

using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using CSV
using DataFrames
using Dates
using Statistics

## Read, reorganise, and normalise the raw ABM demand data.

Essentially, the demand profiles here are normalised using their yearly sums.
Whether this is the way WP5 eventually wants this data is still a bit unclear,
but at least scaling and rescaling should be easy based on these.

Furthermore, scaled normalised values are calculated based on the average yearly
demands across all available weather years.
This approach aims to preserve variability between weather years, while still allowing them to be scaled to match future projections.

Essentially, we're sort of emulating what Hotmaps projections do, using average HDD and CDD from 2002-2012 as the basis for their heating and cooling demand calculations.
However, actual HDD and CDD-based scaling suffers from data availability issues outside EU like so many other data sources.
This is not perfect, but hopefully it will do.

In [ ]:
## Config for normalising the raw data

years = [1995, 2008, 2009, 2012, 2015]
dem_paths = [
    year => "input-data/abm-raw-data/ideal_demands_$(year).csv"
    for year in years
];

In [ ]:
## Read, organise, and normalise demands data.

# Read and organise the data.
dem_cols = [:timestamp, :weather_year, :country, :category, :demand, :value]
dem_data = DataFrame()
for (year, path) in dem_paths
    df = stack(DataFrame(CSV.File(path; header=[1,2])))
    s = split.(df[!, :variable], '_')
    df[!, :country] = String.(getindex.(s, 1))
    df[!, :category] = String.(getindex.(s, 2))
    df[!, :demand] = String.(getindex.(s, 6))
    df[!, :weather_year] .= year
    rename!(
        df,
        :building_archetype_building_process => :timestamp,
    )
    append!(dem_data, df[!, dem_cols])
end

# Add normalised data
dem_sums = combine( # Calculate yearly sums
    groupby(
        dem_data,
        dem_cols[2:end-1] # Skip timestamp and value.
    ),
    :value => sum
)
dem_sums_means = combine( # Calculate mean of yearly sums
    groupby(
        dem_sums,
        dem_cols[3:end-1] # Group by :country, :category, and :demand
    ),
    :value_sum => mean
)
leftjoin!(dem_data, dem_sums, on=dem_cols[2:end-1])
leftjoin!(dem_data, dem_sums_means, on=dem_cols[3:end-1])
dem_data[!, :value_norm] = dem_data.value ./ dem_data.value_sum
dem_data[!, :scaling_factor] = dem_data.value_sum ./ dem_data.value_sum_mean
dem_data[!, :value_norm_scaled] = dem_data.value_norm .* dem_data.scaling_factor
describe(dem_data)

In [ ]:
## Check that normalisation worked as intended

dem_normalised_sums = combine(
    groupby(
        dem_data,
        dem_cols[2:end-1]
    ),
    :value_norm => sum
)
all(dem_normalised_sums.value_norm_sum .≈ 1)

In [ ]:
## Fetch the ratios of peak demand to the yearly demand

dem_normalised_peak_demands = combine(
    groupby(
        dem_data,
        [:country, :demand]
    ),
    :value_norm => maximum
)
describe(dem_normalised_peak_demands)

# These can be used for estimating heat-only boiler capacity later.

## Read, organise, and process Hotmaps scenario data for demand scaling.

The next step is to read the Hotmaps scenario data and process it to a format for easy scaling.


### Norway and Switzerland missing...

Turns out Norway and Switzerland are in fact **not** included in the Hotmaps scenario data,
despite their indices showing up in the .csv files.
They have `missing` data, so no actual numbers for the heating and cooling demand.
Guess I'll have to use neighbouring countries scaled using population as the basis again...

In [ ]:
## Configure reading Hotmaps data

hm_current_path = "input-data/scen_current_building_demand/data/scen_current_building_demand.csv"
hm_ambitious_path = "input-data/scen_ambitious_building_demand/data/scen_ambitious_building_demand.csv";

In [ ]:
## Read and organise the Hotmaps data

hm_current_data_raw = dropmissing!(DataFrame(CSV.File(hm_current_path)))
hm_ambitious_data_raw = dropmissing!(DataFrame(CSV.File(hm_ambitious_path)))
hm_data = vcat(hm_current_data_raw, hm_ambitious_data_raw)
rename!( # Column renaming for dataset compatibility.
    hm_data,
    Dict(
        :NUTS0_code => :country,
        :Year => :scenario_year,
        :Value => :scenario_value,
        :Unit => :unit, 
    )
)
replace!( # Country renaming to align datasets.
    hm_data[!, :country],
    "GB" => "UK",
    "GR" => "EL"
)
supertype_to_category_map = Dict( # Map supertype to category
    "Residential" => "res",
    "Nonresidential_private" => "nonres",
    "Nonresidential_public" => "nonres"
)
hm_data[!, :category] = map(
    x -> get(supertype_to_category_map, x, missing),
    hm_data[!, :Supertype]
)
type_to_demand_map = Dict( # Map types to demands, drop "auxiliary energy demand"
    "space heating" => "heating",
    "cooling" => "cooling",
    "hot water" => "DHW"
)
hm_data[!, :demand] = map(
    x -> get(type_to_demand_map, x, missing),
    hm_data[!, :Type]
)
dropmissing!(hm_data)


## Add missing data for Norway and Switzerland based on Sweden and Austria respectively.

extrapolation_mappings = Dict( # Map Norway and Switzerland to existing data with population-based coefficients
    "NO" => ("SE", 0.52),
    "CH" => ("AT", 0.97)
)
for (c1, (c2, coeff)) in extrapolation_mappings
    df = filter(r -> r.country == c2, hm_data)
    df.country .= c1
    df[!, :scenario_value] .*= coeff
    append!(hm_data, df)
end
describe(hm_data)

In [ ]:
## Aggregate total distributed heating demands across the desired dims.

agg_cols = [:Scenario, :scenario_year, :country, :category, :demand, :unit]
hm_sums = combine(
    groupby(
        filter(r -> r.Fuel != "District heating", hm_data),
        agg_cols
    ),
    :scenario_value => sum
)
hm_sums.Fuel .= "distributed heating"

# Include district heating rows
dh_sums = combine(
    groupby(
        filter(r -> r.Fuel == "District heating", hm_data),
        agg_cols
    ),
    :scenario_value => sum
)
dh_sums.Fuel .= "district heating"
hm_sums = vcat(hm_sums, dh_sums)
describe(hm_sums)

In [ ]:
## Check heating-to-DHW ratios

heating_to_dhw = filter(row -> row.demand == "heating", hm_sums)
dhw = filter(row -> row.demand == "DHW", hm_sums)
heating_to_dhw.scenario_value_sum ./= dhw.scenario_value_sum
filter(row ->  row.country == "FI", heating_to_dhw)

In [ ]:
## Check res-to-nonres ratios

res_to_nonres = filter(row -> row.category == "res", hm_sums)
nonres = filter(row -> row.category == "nonres", hm_sums)
res_to_nonres.scenario_value_sum ./= nonres.scenario_value_sum
filter(row ->  row.country == "FI", res_to_nonres)

## Examine heat pump COPs

The Hotmaps data has separate values for `Electricity` and `Ambient Heat` as the `Fuel` for heat pumps. Let's see how these assumptions vary between countries and end-uses.

In [ ]:
## Examine the heat pump COPs based on their electricity and ambient heat consumption

df = hm_data[hm_data.Technology .== "heat pumps", :]
df = unstack(df, :Fuel, :scenario_value)
df = df[df.Electricity .> 0, :]
df.COP .= (df.Electricity + df[!, Symbol("ambient heat")]) ./ df.Electricity
#sort!(df, :COP; rev=true)
describe(df)

Well that's a bit alarming: The mean COP in the Hotmaps data is above six, which seems pretty unrealistic. Furthermore, the maximum COP is infeasibly high, and weirdly seems to occur mostly for DHW.

Therefore, there's a risk that the modelled heat pump capacity is not enough to cover the demand in certain countries if I use the `Electricity` fuel as the basis for the assumed existing capacity.

In [ ]:
# Let's check space heating and DHW separately:

df_cop_demand = unstack(df, :demand, :COP)
describe(df_cop_demand)

Surprisingly, the space heating COPs are lower than DHW COPs, which shouldn't be the case. Not really sure what's going on here with the Hotmaps data. Ultimately, it might be necessary to estimate the existing heat pump capacity based on the total and the assumed technology parameter COP, but we'll see.

## Estimate generation capacities

We'll have to estimate the technology capacities based on the yearly demands and the estimated demand peak ratio.

In [ ]:
## Calculate estimated peak capacities.

hm_capacity_data = leftjoin( # Combine estimated peak demand ratio data with yearly demands.
    hm_data,
    dem_normalised_peak_demands;
    on=[:country, :demand],
)
rename!(hm_capacity_data, :value_norm_maximum => :peak_to_yearly_demand_ratios_GW_GWh)
hm_capacity_data.capacity_MW = ( # Calculate the estimated capacity in MW
    hm_capacity_data.scenario_value
    .* hm_capacity_data.peak_to_yearly_demand_ratios_GW_GWh
    .* 1000
)
describe(hm_capacity_data)

### Technology mapping

Map the estimated peak capacities to the desired technologies.

In [ ]:
## Figure out heating system mappings from Hotmaps to the desired techs.

heat_techs = unique(hm_data.Technology)
gas_boiler = filter(x -> contains(lowercase(x), "gas boiler"), heat_techs)
bio_boiler = filter(x -> contains(lowercase(x), "biomass"), heat_techs)
oil_boiler = filter(x -> contains(lowercase(x), "oil boiler"), heat_techs)
air_heatpump = filter(x -> contains(lowercase(x), "heat pump"), heat_techs) # Data doesn't distinguish between different heat pumps.
ground_heatpump = filter(x -> contains(lowercase(x), "heat pump"), heat_techs) # Data doesn't distinguish between different heat pumps.
solar_heating = filter(x -> contains(lowercase(x), "solar"), heat_techs)
electric_heating = filter(x -> contains(lowercase(x), "electric"), heat_techs)
district_heating = filter(x -> contains(lowercase(x), "district heating"), heat_techs)

In [ ]:
# Map hm_data technologies to the desired heating techs.

distributed_tech_mapping = Dict( # (Set of technologies, assumed share)
    "gas-boiler" => (gas_boiler, 1.0),
    "bio-boiler" => (bio_boiler, 1.0),
    "oil-boiler" => (oil_boiler, 1.0),
    "air-heatpump" => (air_heatpump, 0.7), # Assumed 70% market share for A2WHPs
    "ground-heatpump" => (ground_heatpump, 0.3), # Assumed 30% market share for G2WHPs
    "solar-heating" => (solar_heating, 1.0),
    "electric-heating" => (electric_heating, 1.0),
    "district-heating" => (district_heating, 1.0)
)

In [ ]:
# Sum the capacities together

rename_cols = Dict(
    :Scenario => :scenario,
    :scenario_year => :scenario_year,
    :country => :country,
    :category => :building_category,
    :output_technology => :technology,
    :demand => :demand,
    :weighted_capacity_MW => :capacity,
    :capacity_unit => :unit,
    :demand_category => :demand_category,
)
heating_capacity_data = DataFrame()
for (name, (techs, weight)) in distributed_tech_mapping
    # Filter relevant technologies.
    df = filter(
        r -> r.Technology in techs && r.Fuel != "ambient heat", # Heat pumps have two fuel rows, omitting "ambient heat" since it's likely not what Alvaro wants.
        hm_capacity_data
    )
    if isempty(df) # Skip the rest of the loop if df is empty
        continue
    end
    # Calculate the weighted capacities
    df.output_technology .= name
    df.weighted_capacity_MW .= df.capacity_MW .* weight
    df.capacity_unit .= "MW"
    df.demand_category .= (name == "district-heating" ? "district heating" : "distributed heating")
    # Final formatting
    df = df[!, collect(keys(rename_cols))] # Drop unused columns
    rename!(df, rename_cols)
    country_cols = Symbol.(unique(df.country))
    df = stack( # Avoid nonresidential private vs nonresidential public duplicate row hassle by unstack-stack summing.
        unstack(
            df,
            :country,
            :capacity;
            combine=sum
        ),
        country_cols;
        variable_name=:country,
        value_name=:capacity
    )
    append!(heating_capacity_data, df)
end
describe(heating_capacity_data)

## Combine datasets, calculate scaled demand, and export desired timeseries.

In [ ]:
## Export timeseries

# Export config
years = [1995, 2008, 2009, 2012, 2015]
dgts = 4 # Number of digits when rounding exports
categories = ["res", "nonres"]
demands = ["heating", "cooling", "DHW"]
index_cols = [:timestamp, :country]

# Export demand timeseries csvs
for cat in categories
    for dem in demands
        for year in years
            df = filter(
                r -> r.weather_year == year && r.category == cat && r.demand == dem,
                dem_data
            )
            df = df[!, vcat(index_cols, :value_norm_scaled)]
            df.value_norm_scaled = round.(df.value_norm_scaled .* 1000; digits=dgts) # Scaling to MW/GWh
            df = sort!(unstack(df[!, vcat(index_cols, :value_norm_scaled)], :country, :value_norm_scaled))
            CSV.write("output/demand-timeseries/$(dem)_$(cat)_wy$(year)_normalised_MW_GWh.csv", df)
        end
    end
end

In [ ]:
## Export country-wise peak-to-yearly-demand ratios

# Config
dgts = 4 # Digits for rounding peak demands

# Export
export_df = sort(dem_normalised_peak_demands)
export_df.value_norm_maximum = round.(
    export_df.value_norm_maximum .* 1000; digits=dgts
) # Round and convert to MW/GWh
export_df.unit .= "MW/GWh"
export_df = unstack(export_df, :country, :value_norm_maximum)
CSV.write(
    "output/peak_to_yearly_demand_ratios_MW_GWh.csv",
    export_df
)

In [ ]:
## Export scenario demands

# Config
dgts = 2 # Number of digits when rounding exports.

# Export table
hm_export = rename(
    hm_sums,
    Dict(
        :Scenario => :scenario,
        :category => :building_category,
        :Fuel => :demand_category,
    )
)
hm_export = sort(hm_export[!, [:scenario, :scenario_year, :country, :building_category, :demand_category, :demand, :unit, :scenario_value_sum]])
hm_export.scenario_value_sum = round.(hm_export.scenario_value_sum; digits=dgts)
hm_export = unstack(hm_export, :country, :scenario_value_sum)
CSV.write("output/scenario_total_yearly_demands_GWh.csv", hm_export)

In [ ]:
## Export assumed existing capacity data

# Config
dgts = 2 # Number of digits when rounding exports.
export_cols = [
    :scenario,
    :scenario_year,
    :country,
    :building_category,
    :demand_category,
    :demand,
    :technology,
    :unit,
    :capacity
]

# Export table
capacity_export = heating_capacity_data[:, export_cols]
capacity_export.capacity = round.(capacity_export.capacity; digits=dgts)
capacity_export = sort!(unstack(
    capacity_export,
    :country,
    :capacity,
    combine=sum # Avoid nonresidential private vs nonresidential public duplicate row hassle.
))
CSV.write("output/scenario_estimated_existing_capacities_MW.csv", capacity_export)

## Plot diagnostics to see how the data looks

In [ ]:
## Scale demands based on select scenario year.

scenario_year = 2025 # Select scenario year
scenario = "current" # Select scenario

scen_data = filter( # Filter the desired scenario and year
    r -> r.scenario_year == scenario_year && r.Scenario == scenario,
    hm_sums
)
scen_data = combine( # Sum "district heating" and "distributed heating"
    groupby(scen_data, [:country, :category, :demand]),
    :scenario_value_sum => sum
)
full_data = leftjoin(dem_data, scen_data, on=[:country, :category, :demand])
full_data.final_scaled_value_MW = full_data.value_norm_scaled .* full_data.scenario_value_sum_sum .* 1000
describe(full_data)

In [ ]:
## Plot diagnostics

using StatsPlots

plot_year = 1995
plot_cols = [:timestamp, :category, :final_scaled_value_MW]
countries = Set(full_data.country)
f = nothing # How many timesteps to plot, `nothing` plots all

for c in countries
    for dem in demands
        df = filter(
            r -> r.weather_year == plot_year && r.demand == dem && r.country == c,
            full_data
        )
        df = df[!, plot_cols]
        df = unstack(df, :category, :final_scaled_value_MW)
        isnothing(f) ? nothing : df = first(df, f)
        plt = plot(
            df[!, 1],
            hcat(df[!, 2], df[!, 3]);
            title = "$(c) $(dem)",
            labels = reshape(names(df)[2:3], 1, 2)
        )
        display(plt)
    end
end

In [ ]:
## plot When2Heat data for comparison

f = 168
w2h_data = DataFrame(CSV.File("input-data/when2heat/spaceHeat_2025_hour_country_1995.csv"))
for c in intersect(Set(w2h_data.country), Set(full_data.country))
    # ABM data
    df = filter(
        r -> r.weather_year == 1995 && r.demand == "heating" && r.country == c,
        full_data
    )
    df = df[!, plot_cols]
    df = combine(
        groupby(df, :timestamp),
        :final_scaled_value_MW => sum
    )
    df.final_scaled_value_MW_sum ./= maximum(df.final_scaled_value_MW_sum)
    isnothing(f) ? nothing : df = first(df, f)
    plt = plot(
        df[!, 1],
        df[!, 2];
        title = "$(c)",
        label = "abm"
    )

    # When2Heat data
    df2 = filter(r -> r.country == c, w2h_data)
    isnothing(f) ? nothing : df2 = first(df2, f)
    plot!(
        plt,
        df[!, 1],
        df2[!, 3];
        label = "w2h"
    )
    display(plt)
end

**Not great, not terrible.**

There's a pretty significant difference between the heating demands from
ArchetypeBuildingModel and the when2heat datasets.
In a sense, it's not that surprising, as the they use wildly different
approaches for calculating the heating demands.

Overall, I would still expect when2heat data to be more reliable,
since from what I understand it is a heating-degree-day-approach
based on actual gas-use measurements from Germany.
However, I'm a bit sceptical on how well the when2heat timeseries apply
to countries with different heating preferences _(e.g. the Nordics)_.

Not sure if ArchetypeBuildingModel really should even be used for
something like this, but for now it seems to be what we're stuck with.